1. preparar la data
2. leer dataset
input(x)--> Comentarios(review)
output(y)--> sentimientos

In [41]:
import pandas as pd


In [42]:
df_review = pd.read_csv('IMDB_Dataset.csv')

In [43]:
#muestra de 10 mil
df_positivo = df_review[df_review['sentiment']=='positive'][:9000]
df_negativo = df_review[df_review['sentiment']=='negative'][:1000]

df_review_des = pd.concat([df_positivo, df_negativo])
df_review_des.value_counts('sentiment')


sentiment
positive    9000
negative    1000
Name: count, dtype: int64

Data desbalanceado

In [44]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler()
df_review_bal, df_review_bal['sentiment'] = rus.fit_resample(df_review_des[['review']], df_review_des['sentiment']) #necesita 2D
df_review_bal.value_counts('sentiment')

sentiment
negative    1000
positive    1000
Name: count, dtype: int64

SEPARANDO DATA PARA ENTRENAR(train) Y DATA PARA TESTEAR(test)


In [45]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df_review_bal, test_size=0.33, random_state=42)

In [46]:
train_x, train_y = train['review'], train['sentiment']
test_x, test_y = test['review'], test['sentiment']


las maquinas funcionan mejor con datanumerica.
representacion de texto (Bag of text)

In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
train_x_vector = tfidf.fit_transform(train_x) #tfid  obtiene los mejores parametros y transform aplica esosparametros
test_x_vector = tfidf.transform(test_x)
test_x_vector


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 53591 stored elements and shape (660, 20506)>

suport vector machine (SVM)

In [48]:
from sklearn.svm import SVC

In [49]:
svc = SVC(kernel='linear')
svc.fit(train_x_vector, train_y)

SVC(kernel='linear')

In [50]:
#testeo
print(svc.predict(tfidf.transform(['A good movie'])))
print(svc.predict(tfidf.transform(['An excellent movie'])))
print(svc.predict(tfidf.transform(['"I did not like tis movie at all I gave this movie away"'])))


['positive']
['positive']
['negative']


arbol de decision: se usa para construir un modelo que puede pedecir la clase o el valor del modelo objetivo 

In [51]:
from sklearn.tree import DecisionTreeClassifier
dec_tree = DecisionTreeClassifier()
dec_tree.fit(train_x_vector, train_y)

DecisionTreeClassifier()

In [52]:
#naive bayes
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(train_x_vector.toarray(), train_y)

GaussianNB()

regresion logistica

In [53]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(train_x_vector, train_y)

LogisticRegression()

EVALUACION DE LOS MODELOS Score(no es la prescision)

In [58]:
print(svc.score(test_x_vector, test_y))
print(dec_tree.score(test_x_vector, test_y))
print(gnb.score(test_x_vector.toarray(), test_y))
print(lr.score(test_x_vector, test_y))

0.8409090909090909
0.696969696969697
0.5984848484848485
0.8393939393939394


In [ ]:
#F1 score 2*(Recall*Precision)/(Recall+Precision) valor minimo 0, valor maximo 1
from sklearn.metrics import f1_score
f1_score(test_y, svc.predict(test_x_vector),
            labels=['positive', 'negative'], average=None)


array([0.84935438, 0.83146067])

Reporte de clasificacion

In [61]:
from sklearn.metrics import classification_report
print(classification_report(test_y, svc.predict(test_x_vector),
            labels=['positive', 'negative']))


              precision    recall  f1-score   support

    positive       0.82      0.88      0.85       335
    negative       0.87      0.80      0.83       325

    accuracy                           0.84       660
   macro avg       0.84      0.84      0.84       660
weighted avg       0.84      0.84      0.84       660



In [62]:
#confusion amtrix: revela verdaderos positivos/negativos falsos positivos/negativos
from sklearn.metrics import confusion_matrix
print(confusion_matrix(test_y, svc.predict(test_x_vector),
            labels=['positive', 'negative']))


[[296  39]
 [ 66 259]]


optimizacion de modelo

In [63]:
#gridsearchCV
from sklearn.model_selection import GridSearchCV
parameters = {'C':[1,4,8,16,32], 'kernel':['linear', 'rbf']}
svc = SVC()
svc_grid = GridSearchCV(svc, parameters, cv=5)
svc_grid.fit(train_x_vector, train_y)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [1, 4, 8, 16, 32], 'kernel': ['linear', 'rbf']})

In [65]:
print(svc_grid.best_estimator_)
print(svc_grid.best_params_)
print((svc_grid.best_score_))

SVC(C=1, kernel='linear')
{'C': 1, 'kernel': 'linear'}
0.8373134328358208
